In [9]:
import numpy as np
import matplotlib
import nltk
import os
import random
import re
import torch

from torch.utils.data import TensorDataset
from utils import *

random.seed(42)

%matplotlib inline

In [2]:
# define experiment
exp='/exp_1a'

In [3]:
USE_CUDA = True

In [5]:
# load dataset into memory, and get w2idx, idx2w, w2freq dictionaries and lists of input and output sentences
cmd_vocab, w2i_cmds, i2w_cmds, cmds, act_vocab, w2i_acts, i2w_acts, acts = load_dataset(exp=exp, split='/train')

In [6]:
# create input and output language pairs
cmd_act_pairs = create_pairs(cmds, acts)

In [7]:
# show random command-action pair
random_pair = random.choice(cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

Command: ['look', 'left', 'thrice', 'after', 'turn', 'opposite', 'left']
Action: ['I_TURN_LEFT', 'I_TURN_LEFT', 'I_TURN_LEFT', 'I_LOOK', 'I_TURN_LEFT', 'I_LOOK', 'I_TURN_LEFT', 'I_LOOK']


In [8]:
cmd_act_pair = pairs2idx(random_pair, w2i_cmds, w2i_acts)
print("Command sequence: {}".format(cmd_act_pair[0]))
print("Action sequence: {}".format(cmd_act_pair[1]))

Command sequence: tensor([12,  4,  6,  9, 15,  7,  4,  1])
Action sequence: tensor([0, 4, 4, 4, 8, 4, 8, 4, 8, 1])
